In [0]:
!pip install fire

     |████████████████████████████████| 81kB 2.9MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=f51d518052a40b1572a5063ea47323c627ac99362bb4172509871275c47d51e3
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My\ Drive/"
base_dir = root_dir + 'fastai-v3/les8'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/fastai-v3/les8

/content/gdrive/My Drive/fastai-v3/les8


In [0]:
pwd

'/content/gdrive/My Drive/fastai-v3/les8'

In [0]:
#export
from exp.nb_07 import *

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


Layerwise Sequential Unit Variance (LSUV)

Getting the MNIST data and a CNN


In [0]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [0]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [0]:
nfs = [8,16,32,64,64]

In [0]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
    
    def forward(self, x): return self.relu(self.conv(x))
    
    @property
    def bias(self): return -self.relu.sub
    @bias.setter
    def bias(self,v): self.relu.sub = -v
    @property
    def weight(self): return self.conv.weight

In [0]:
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)



Now we're going to look at the paper All You Need is a Good Init, which introduces Layer-wise Sequential Unit-Variance (LSUV). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:


In [0]:
run.fit(2, learn)

train: [1.95383890625, tensor(0.3068, device='cuda:0')]
valid: [0.720376416015625, tensor(0.7577, device='cuda:0')]
train: [0.3832821484375, tensor(0.8783, device='cuda:0')]
valid: [0.1469978759765625, tensor(0.9549, device='cuda:0')]


Now we recreate our model and we'll try again with LSUV. Hopefully, we'll get better results!

In [0]:
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Helper function to get one batch of a given dataloader, with the callbacks called to preprocess it.

In [0]:
#export
def get_batch(dl, run):
    run.xb,run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb,run.yb

In [0]:
xb,yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use sum(list, []) to concatenate the lists the function finds (sum applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [0]:
#export
def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

In [0]:
mods = find_modules(learn.model, lambda o: isinstance(o,ConvLayer))

In [0]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

This is a helper function to grab the mean and std of the output of a hooked layer.

In [0]:
def append_stat(hook, mod, inp, outp):
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()

In [0]:
mdl = learn.model.cuda()

So now we can look at the mean and std of the conv layers of our model.

In [0]:
with Hooks(mods, append_stat) as hooks:
    mdl(xb)
    for hook in hooks: print(hook.mean,hook.std)

0.3893089294433594 0.7792686223983765
0.3996789753437042 0.6620614528656006
0.4108506143093109 0.643345057964325
0.3801940083503723 0.5647823214530945
0.24098142981529236 0.38366857171058655


We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The mdl(xb) is not None clause is just there to pass xb through mdl and compute all the activations so that the hooks get updated.

In [0]:
#export
def lsuv_module(m, xb):
    h = Hook(m, append_stat)

    while mdl(xb) is not None and abs(h.mean)  > 1e-3: m.bias -= h.mean
    while mdl(xb) is not None and abs(h.std-1) > 1e-3: m.weight.data /= h.std

    h.remove()
    return h.mean,h.std

We execute that initialization on all the conv layers in order:

In [0]:
for m in mods: print(lsuv_module(m, xb))

(0.11027354747056961, 1.0)
(0.1490212231874466, 1.0)
(0.10891561210155487, 1.0000001192092896)
(0.16913190484046936, 0.9999999403953552)
(0.27742981910705566, 1.000000238418579)




Note that the mean doesn't exactly stay at 0. since we change the standard deviation after by scaling the weight.

Then training is beginning on better grounds.


In [0]:
%time run.fit(2, learn)

train: [0.499746171875, tensor(0.8388, device='cuda:0')]
valid: [0.12449700927734375, tensor(0.9637, device='cuda:0')]
train: [0.102100068359375, tensor(0.9678, device='cuda:0')]
valid: [0.1132468505859375, tensor(0.9658, device='cuda:0')]
CPU times: user 3 s, sys: 1.17 s, total: 4.16 s
Wall time: 4.19 s


LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

In [0]:
!python notebook2script.py 07a_lsuv.ipynb

Converted 07a_lsuv.ipynb to exp/nb_07a.py
